### KAYAK

Dans ce notebook, nous verrons toute la partie **Scraping de données** pour récupérer les informations concernant les hotels qui se trouvent dans les villes que nous avons choisies d'étudier.

Il y a plusieurs techniques de Data Mining : 
* soit on utilise la librairie `requests` pour le faire mais elle est assez limitée
* soit on peut utiliser `Beautiful Soup` ou `Selenium` qui sont assez complet pour scraper les données 
* soit on peut utiliser `Scrapy` qui est assez complexe mais plus complet que ceux cités avant.

Notre choix se portent donc sur `Scrapy`

Nous allons utiliser l'url de recherche : https://www.booking.com/searchresults.fr.html pour faire nos requètes


In [2]:
import requests
import pandas as pd
import json
import os
from collections import Counter
import numpy as np

### Scraping

/kayak
   |__scrapy.cfg                 # fichier qui definit le projet
   |__/kayak
         |__items.py             # fichier qui definit les models items (type de données)
         |__middlewares.py       # fichier qui definit comment agiront les middlewares
         |__pipelines.py         # fichier qui permet de faire des pipelines de parsing
         |__settings.py          # fichier qui definit tous les settings
         |__ __init__.py          
         |__/spider
               |__ __init__.py   # fichier qui initialise le spider
               |__CONSTANTES.py  # fichier qui regroupe les filtres
               |__hb.py          # fichier du Spider

L'Objectif est de recupérer pleins d'informations sur les hotels grace à 3 methodes successives du Spider.

Notre spider va effectuer 3 étapes pour scraper Booking :

* La 1ere etape consiste à entrer une url de recherche avec le nom de la ville et différents filtres de recherche dans un formulaire avec la methode de Spider : `scrapy.FormRequest.from_response()`

* La 2nde etape va être d'utiliser le callback du FormRequest pour aller sur la page des 25 meilleurs hotels et d'en extraire les informations utiles (nom de l'hotel, sa description, son lien url, sa note / 5, ect...) car chaque hotel est représenté sur cette page par un encadré qui est dans une div `'//div[@data-testid="property-card"]'` et qui regroupe toutes ces informations

* La 3eme etape va permettre de recupérer les coordonnées de chaque hotel en faisant un `response.follow()` pour conserver les informations que l'on vient de parser grace au `meta` du follow() et on appelle la nouvelle methode grace au callback

On effectue ce processus dans uen boucle qui itère sur tous les villes et recupère ainsi 25 hotels par ville.

Il existe une grande quantité de filtres pour affiner ses recherches : chaque critère de recherche est séparé par un `&` dans l'url.

par exemple:

* pour rechercher la ville de Paris, il suffit me mettre le filtre : `&ss='Paris'`
* pour ne choisir que des hotels, il suffit de mettre le filtre : `&dest_type='hotel'`
* pour choisir 3 chambres, on met le filtre : `&no_rooms=3`

On va utiliser comme technique celle qui nous permet de reconstruire une url à partir de filtres pour faire la 1ere étape

In [ ]:
https://www.booking.com/searchresults.fr.html?aid=397594
        &label=gog235jc-1DCAEoggI46AdIDVgDaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4Asyp0ZkGwAIB0gIkZjA4ZGZiMGYtMGJmZC00MTEzLTgxOWYtMGYxNDk1ZWYzMTAx2AIE4AIB
        &sid=9a0ba32de0342ed74b61838fa4ac7c7b
        &sb=1
        &sb_lp=1
        &src=index
        &src_elem=sb
        &error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D397594%26label%3Dgog235jc-1DCAEoggI46AdIDVgDaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4Asyp0ZkGwAIB0gIkZjA4ZGZiMGYtMGJmZC00MTEzLTgxOWYtMGYxNDk1ZWYzMTAx2AIE4AIB%26sid%3D9a0ba32de0342ed74b61838fa4ac7c7b%26sb_price_type%3Dtotal%26%26
        &ss=Paris
        &is_ski_area=0
        &checkin_year=
        &checkin_month=
        &checkout_year=
        &checkout_month=
        &group_adults=2
        &group_children=0
        &no_rooms=1
        &b_h4u_keep_filters=
        &from_sf=1
        &dest_id=
        &dest_type=
        &search_pageview_id=d4b26326b5ea0299
        &search_selected=false

In [ ]:
new_url:
    https://www.booking.com/searchresults.fr.html?aid=397594&ss=Paris

Ceci est le fichier `CONSTANTES.py` qui est dans le meme repertoire que le script du Spider et qui permet de faire plusieurs actions :

* Definir des filtres pour avoir une recherche plus ciblée
* Convertir des caractères spéciaux en caractères normaux

In [ ]:
# fichier CONSTANTES.py

NB_ETOILE = 4                  # 4 etoiles
BUDGET_100_150 = 3             # budget entre 100 et 150
BUDGET_150_200 = 4             # budget entre 150 et 200
SCORE_8_SUR_10 = 80            # note minimum de 8/10 
DISTANCE_CENTRE_VILLE = 3000   # distance moins de 3km du centre ville


cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
          "Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon",
          "Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes",
          "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse",
          "Montauban","Biarritz","Bayonne","La Rochelle"]

d = {"\u00e8" : "e",
 "\u00e9" : "e",
 "\u00ee" : "i",
 "\u00d4" : "O",
 "\u00c2" : "A",
 "\u00b0" : "",
 "\u0153" : "oe",
 "\u00e7" : "c",
 "\u00f2" : "o",
 "\u00ea" : "e",
 "\u00e0" : "a",
 "\u00e9" : "e",
 "\u2019" : "'",
 "\u00e2" : "a",
 "\u00c9" : "E",
 "\u2606" : " ",
 "\u2729" : " ",
 "\u00f4" : "o",
 "\u00e8" : "e",
 "\u00ee" : "i",
 "\u00fb" : "u",
 "\u00e9" : "e",
 "\u00e7" : "c",
 "\u00e2" : "a",
 "\u2122" : ""}

def utf8(dic, string):
    x = ""
    for el in string:
        if el in dic:
            x += dic[el]
        else:
            x += el
    return x

Nous ouvrons le fichier texte contenant une liste de proxy pour eviter de se faire detecter en tant que bot de parsing et nous l'implementerons dans le fichier settings.py

In [26]:
p = r'C:\Users\david\Downloads'
file = os.path.join(p, '', 'proxy.txt')
data = []
with open(file, 'r') as f:
    contents = f.read()
    data.append(contents.replace("\n", ','))

# On obtient toutes les adresses IP pour faire de la rotation de proxy
txt_file = ''.join(data).split(',')[:-1]

Nous mettons un delais entre 2 recupérations de données, nous choisissons un user_agent qui n'est pas celui par default (scrapy) pour ne pas se faire repérer comme bot, ect...

In [ ]:
### setting.py

BOT_NAME = 'kayak'

SPIDER_MODULES = ['kayak.spiders']
NEWSPIDER_MODULE = 'kayak.spiders'



USER_AGENT =  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'


ROBOTSTXT_OBEY = False


DOWNLOAD_DELAY = 3

COOKIES_ENABLED = False


DOWNLOADER_MIDDLEWARES = {
    'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
    'scrapy_user_agents.middlewares.RandomUserAgentMiddleware': 400,
    'kayak.middlewares.KayakDownloaderMiddleware': 543,
}


AUTOTHROTTLE_ENABLED = True

AUTOTHROTTLE_START_DELAY = 5

AUTOTHROTTLE_MAX_DELAY = 60

# proxy.txt
ROTATING_PROXY_LIST = txt_file

In [2]:
import scrapy
import logging
from scrapy.crawler import CrawlerProcess


class HotelsbookingSpider(scrapy.Spider):
    name = 'hb'
    allowed_domains = ['www.booking.com']
    start_urls = ['https://www.booking.com/searchresults.fr.html?ss=']
    
    def parse(self, response):

        cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
              "Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon",
              "Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes",
              "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse",
              "Montauban","Biarritz","Bayonne","La Rochelle"]

            result = (city for city in cities)
            #filters = [f"&nflt=class%3D{cst.NB_ETOILE}", f"%3Bpri%3D{cst.BUDGET_100_150}", f"%3Bpri%3D{cst.BUDGET_150_200}", \
              #f"%3Breview_score%3D{cst.SCORE_8_SUR_10}", f"%3Bdistance%3D{cst.DISTANCE_CENTRE_VILLE}"]

            for i in range(len(cities)):
                yield scrapy.FormRequest.from_response(
                    response,
                    formdata={'ss': next(result)}, # + ''.join([f for f in filters])},
                    callback=self.search
                )

    def search(self, response):
        
        hotels = response.xpath('//div[@data-testid="property-card"]')

        for hotel in hotels:
            link = hotel.xpath('.//h3/a[@data-testid="title-link"]/@href').get()
            title =  hotel.xpath('.//h3/a[@data-testid="title-link"]/div/text()').get("").replace("\u00e8", "e").replace("\u00e9", "e").replace("\u00ee", "i").replace("\u00d4", "O").replace("\u00c2", "A").replace("\u00b0", "").replace("\u0153", "oe").replace("\u00e7", "c").replace("\u00f2", "o").replace("\u00ea", "e").replace("\u00e0", "a").replace("\u00e9", "e").replace("\u2019", "'").replace("\u00e2", "a").replace("\u00c9", "E").replace("\u2606", " ").replace("\u2729", " ").replace('\u00f4', 'o').replace('\u00e8','e').replace("\u00ee", "i").replace("\u00fb", "u").replace("\u00e9", "e").replace("\u00e7", "c").replace("\u00e2", "a").replace("\u2122", "")
            #'title' : cst.utf8(cst.d, hotel.xpath('.//h3/a[@data-testid="title-link"]/div/text()').get(""))
            price =  hotel.xpath('.//div[@data-testid="price-and-discounted-price"]/text()').get()
            location =  hotel.xpath('.//span[@data-testid="address"]/text()').get("")
            score = hotel.xpath('.//div[@data-testid="review-score"]/div/text()').get("")
            review_count = hotel.xpath('.//div[@data-testid="review-score"]/div[2]/div[2]/text()').get("")
            stars = hotel.xpath('.//div[@data-testid="rating-stars"]/span').getall()
            description = hotel.xpath('//div[@class="d8eab2cf7f"]/text()').get()
            image = hotel.xpath('.//img[@data-testid="image"]/@src').get()
                                                                                                                                                                                                                                                                    
            yield response.follow(url=link, callback=self.parse_url, meta={'link' : link, 'title' : title, 'price' : price, 'location' : location, 'score' : score, 'review_count' : review_count, 'stars' : stars, 'description': description, 'image': image})      
            

    def parse_url(self, response):

        link = response.request.meta['link']
        title = response.request.meta['title']
        price = response.request.meta['price']
        location = response.request.meta['location']
        score = response.request.meta['score']
        review_count = response.request.meta['review_count']
        stars = response.request.meta['stars']
        description = response.request.meta['description']
        image = response.request.meta['image']

        coords = response.xpath("//a[@data-atlas-latlng]/@data-atlas-latlng").get(default='')

        yield {
               'link' : link,
               'title' : title,
               'price' :  price,
               'location' : location,
               'score' : score,
               'review_count' : review_count,
               'stars' : stars,
               'description' : description,
               'image' : image,
               'coords' : coords,
               'lat' : coords.split(",")[0],
               'lon' : coords.split(",")[1]
        }


le CrawlerProcess permet de scraper les données directement depuis un notebook : nous stockons les données dans un fichier `cities.json`

In [3]:
# Initializing the crawler process
filename = "cities.json"

if filename in os.listdir():
        os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename: {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(HotelsbookingSpider)
process.start()

2022-09-29 16:24:44 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-09-29 16:24:44 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 3.4.8, Platform Windows-10-10.0.19044-SP0
2022-09-29 16:24:44 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) '
               'Gecko/20100101 Firefox/92.0'}
2022-09-29 16:24:44 [scrapy.extensions.telnet] INFO: Telnet Password: b62cb375b2cfc7bd
2022-09-29 16:24:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle

In [ ]:
df = pd.read_json(filename)

In [112]:
df.shape

(875, 12)

In [ ]:
df.to_csv('./src/filename.csv', index=False)